In [ ]:
# Simulation of the HCW model with MPC control and LQR control
# Selected controllers:
# - Linear MPC with OSQP
# - Convex MPC with Ipopt
# - Convex MPC with ECOS
# - MPPI control
# - CDDP MPC
# - LQR control

# Load packages
# using Pkg
# Pkg.activate("..")
# Pkg.instantiate()

In [1]:
using Revise
using CDDPOld
using GLMakie
using SparseArrays
using JuMP
using OSQP
using Ipopt
using SCS
using ECOS
using ControlSystems
using LinearAlgebra
using StaticArrays
using OrdinaryDiffEq
using CUDA
using GLMakie
using Random
using Distributions

Precompiling CDDPOld
  ✓ IntervalArithmetic → IntervalArithmeticForwardDiffExt
        Info Given CDDPOld was explicitly requested, output will be shown live 
ERROR: LoadError: InitError: UndefVarError: `GR_jll` not defined
Stacktrace:
  [1] __init__()
    @ GR.GRPreferences ~/.julia/packages/GR/SdanO/src/preferences.jl:64
  [2] run_module_init(mod::Module, i::Int64)
    @ Base ./loading.jl:1134
  [3] register_restored_modules(sv::Core.SimpleVector, pkg::Base.PkgId, path::String)
    @ Base ./loading.jl:1122
  [4] _include_from_serialized(pkg::Base.PkgId, path::String, ocachepath::String, depmods::Vector{Any})
    @ Base ./loading.jl:1067
  [5] _require_search_from_serialized(pkg::Base.PkgId, sourcepath::String, build_id::UInt128)
    @ Base ./loading.jl:1581
  [6] _require(pkg::Base.PkgId, env::String)
    @ Base ./loading.jl:1938
  [7] __require_prelocked(uuidkey::Base.PkgId, env::String)
    @ Base ./loading.jl:1812
  [8] #invoke_in_world#3
    @ ./essentials.jl:926 [inlined]
  [9] 

LoadError: The following 1 direct dependency failed to precompile:

CDDPOld [4251257f-8958-45e1-955e-ef645c8547ab]

Failed to precompile CDDPOld [4251257f-8958-45e1-955e-ef645c8547ab] to "/home/tom/.julia/compiled/v1.10/CDDPOld/jl_P8OlG5".
[91m[1mERROR: [22m[39mLoadError: InitError: UndefVarError: `GR_jll` not defined
Stacktrace:
  [1] [0m[1m__init__[22m[0m[1m([22m[0m[1m)[22m
[90m    @[39m [35mGR.GRPreferences[39m [90m~/.julia/packages/GR/SdanO/src/[39m[90m[4mpreferences.jl:64[24m[39m
  [2] [0m[1mrun_module_init[22m[0m[1m([22m[90mmod[39m::[0mModule, [90mi[39m::[0mInt64[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m./[39m[90m[4mloading.jl:1134[24m[39m
  [3] [0m[1mregister_restored_modules[22m[0m[1m([22m[90msv[39m::[0mCore.SimpleVector, [90mpkg[39m::[0mBase.PkgId, [90mpath[39m::[0mString[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m./[39m[90m[4mloading.jl:1122[24m[39m
  [4] [0m[1m_include_from_serialized[22m[0m[1m(